# Case 2: Satellite navigation


Global navigation satellite systems (GNSS) determine a user's position using a receiver (such as a cell phone) and satellites orbiting the Earth. The most widely used GNSS is the Global Positioning System (GPS).

In GNSS, the position of the device is determined by measuring the time difference between when a signal is sent from the satellite and when it is received by the receiver.

Multiplying the time difference with the speed of light, $c=299 792 458 $m/s, one gets the distance between each satellite and the receiver.

## Matematical formulation

All coordinates are given as vectors in $\mathbf{R}^3$. Specifically, we use a coordinate system with origin at the center of the Earth, $z$-axis pointing towards the North pole, $x$-axis pointing towards $0^\circ N, 0^\circ E$, and $y$-axis pointing towards $0^\circ N, 90^\circ E$.

![Earth centered coordinate system](Earth_Centered_Coordinate_System.png)

Known parameters:

* Coordinates of the satellites: at the time of measurement
$\vec{v}^1=\begin{bmatrix} v^1_x\\ v^1_y \\ v^1_z\end{bmatrix}, \vec{v}^2, \dotsc, \vec{v}^k$
* Measured distances:
$ R^1, R^2, \dotsc R^k$ - Measured time differences multiplied by speed of light

Unknowns:
* Position of the receiver:
$\vec{x}=[x,y,z]^T$.
* Clock error of the receiver $ c \Delta t$.

The satellites carry very precise atomic clocks, the clock on the receiver is not as accurate. Therefore we expect that the clock of the receiver may be wrong, and correct for it with the unknown $\Delta t$. We multiply the clock error with $c$, so that all unknown quantities are measured in meters.

The equations connecting the unknowns with the known parameters are:
$$
\begin{aligned}
R^j & = \|\vec{x}-\vec{v}^j\| + c\Delta t & j=1,2,\dotsc, k\\
    &= \sqrt{ (x-v^j_x)^2+(y-v^j_y)^2+(z-v^j_z)^2} +c\Delta t &  j=1,2,\dotsc, k
\end{aligned}
$$

The equations are non-linear in $(x,y,z)$. We linearize around a fixed position (preferably nearby) $\vec{x}^0$, and set 
$$
\begin{aligned}
\begin{bmatrix}
x\\y\\z
\end{bmatrix}
& =
\begin{bmatrix}
x^0\\y^0\\z^0
\end{bmatrix}
+
\begin{bmatrix}
\Delta x\\\Delta y\\\Delta y
\end{bmatrix}\\
\vec{x} & = \vec{x}^0+\Delta \vec{x}.
\end{aligned}
$$

The linear approximation is
$$
R^j \approx \rho^j_0 + \frac{x^0-v^j_x}{\rho^j_0} \Delta x+ \frac{y^0-v^j_y}{\rho^j_0} \Delta y + \frac{z^0-v^j_z}{\rho^j_0} \Delta z + c\Delta t,
$$
where
$\rho^j_0 = \sqrt{ (x^0-v^j_x)^2+(y^0-v^j_y)^2+(z^0-v^j_z)^2}$ is the distance between satellite $j$ and the fixed position $\vec{x}^0$.



Ignoring the approximation, we get, for each satellite, a linear equation in the four unknowns $\Delta x, \Delta y, \Delta z, c\Delta t$

$$
\begin{bmatrix} \frac{x^0-v^j_x}{\rho^j_0} & \frac{y^0-v^j_y}{\rho^j_0} & \frac{z^0-v^j_z}{\rho^j_0} & 1 \end{bmatrix}
\begin{bmatrix}
\Delta x \\
\Delta y \\
\Delta z \\ 
c\Delta t\end{bmatrix}
=  R^j-\rho^j_0.
$$

For simpler notation, we define
$$
X^j=\frac{x^0-v^j_x}{\rho^j_0}\quad Y^j = \frac{y^0-v^j_y}{\rho^j_0} \quad Z^j= \frac{z^0-v^j_z}{\rho^j_0}
$$
Usually (and preferably), we have measurements for more than four satellites. For example, if there are six satellites visible, the system of equations is

$$
\underbrace{
\begin{bmatrix}
X^1 & Y^1 & Z^1 & 1\\
X^2 & Y^2 & Z^2 & 1\\
X^3 & Y^3 & Z^3 & 1\\
X^4 & Y^4 & Z^4 & 1\\
X^5 & Y^5 & Z^5 & 1\\
X^6 & Y^6 & Z^6 & 1
\end{bmatrix}}_
{A}
\underbrace{
\begin{bmatrix} 
\Delta x\\ \Delta y \\ \Delta z \\ c\Delta t
\end{bmatrix}}
_{\Delta \vec{x}}
=
\underbrace{
\begin{bmatrix}
R^1-\rho^1_0\\
R^2-\rho^2_0\\
R^3-\rho^3_0\\
R^4-\rho^4_0\\
R^5-\rho^5_0\\
R^6-\rho^6_0\\
\end{bmatrix}}_{\vec{b}}
$$


The system is overdetermined (more equations than unknowns), so an exact solution is not possible. Instead, we find a least-squares solution.

Thanks to Ola Øvstedal (GMPE240) for the problem and data.


## Example data:

In [2]:
import numpy as np

c=299792458

v = np.array([[17320278.617,  -8377650.251, 18417718.933], # satellite positions
              [23903923.512,   -575354.301, 12082414.203],
              [18145496.573,    573199.862, 19274924.497],
              [ 4943294.735, -14506778.876, 21432563.831],
              [13257395.396,  20015827.354, 11298737.140],
              [10229840.139,  10977541.920, 21818177.084]])

R = np.array([21166927.434, # measured distance to each satellite
              21787043.623, 
              20356882.560, 
              22039726.386,
              22639167.771, 
              20600717.361]) 

x0 = np.array([3174160., 598860., 5481410.]) # Fixed point

In [3]:
def distances(x, v): #computes the distances between a point x and the satellite positions stored in v
    return np.linalg.norm(x-v, axis=1)
    
rho0 = distances(x0,v)
rho0

array([21166919.87565875, 21787040.43637279, 20356881.37715792,
       22039724.86450164, 22639159.70903579, 20600718.90438   ])

### Task 1: Compute the matrix $A$ from above

In [4]:
def dmatrix(x0,v):
    # fill in your code her
    X=(x0-v)/distances(x0,v)[:, None]
    return np.hstack((X, np.ones((6,1))))
A= dmatrix(x0,v)
print(A)

[[-0.66831257  0.42408203 -0.61115689  1.        ]
 [-0.95147221  0.05389508 -0.30297847  1.        ]
 [-0.73544352  0.00126051 -0.67758485  1.        ]
 [-0.08027027  0.68538237 -0.7237456   1.        ]
 [-0.44538912 -0.85767173 -0.25695862  1.        ]
 [-0.34249679 -0.50380193 -0.79301927  1.        ]]


### Task 2: solve the least square problem $A\Delta\vec{x}=\vec{b}$ (using any method you have learned)


In [5]:
Q,RR=np.linalg.qr(A)
dx=np.linalg.solve(RR, Q.T@(R-rho0))
print(dx)

[ 1.7779709   1.53415211 13.96994947 12.16934314]


In [6]:
# Compute estimated position
x= x0+dx[0:3]
print(x)

[3174161.7779709   598861.53415211 5481423.96994947]


In [7]:
# estimated time error:
print(dx[3]/c)

4.059255932485559e-08


Where is it? Converting from cartesian coordinates to standard longitude, latitude and elevation is not trivial, below is one implementation. See https://en.wikipedia.org/wiki/Geographic_coordinate_conversion



In [8]:


def XYZtolatlonelev(position):
    # computes the latitude, longitude and elevation (over the ellipsoid) corresponding to position
    
    a =  6378137.0 # Earth equatorial radius
    b =  6356752.3 # Earth polar radius
    esq =  1-(b/a)**2 # squared eccentricity of earth
    
    x=position[0]
    y=position[1]
    z=position[2]

    p =  np.sqrt(x**2+y**2)
    
    coeffs =[(1-esq)*z**2, -2*(1-esq)*z**2, p**2+(1-esq)*z**2-esq**2*a**2, -2*p**2, p**2]
    kappa=np.roots(coeffs)[0].real
    
    lon = np.rad2deg(np.arctan2(y,x))
    lat= np.rad2deg(np.arctan(z*kappa/p))
    elev = np.sqrt(p**2+z**2*kappa**2)*(1/kappa-(1-esq))/esq
    
    
    return lat, lon, elev
    

In [9]:
XYZtolatlonelev(x)

(59.65753884398726, 10.684268972296454, 141.56579202807862)

[Link to the coordinates](https://www.norgeskart.no/#!?project=norgeskart&sok=59.65753884398726,%2010.684268972296454)